In [17]:
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Conv2D, Flatten, Dense
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras import backend as K
from keras import initializers
import keras

import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os import system
import os
import random

import imageio

In [22]:
np.set_printoptions(suppress=True)

img_width, img_height = 31, 31

nb_epoch = 50
batch_size = 32

In [3]:
base_dir = '/scratch/image_datasets/1_for_learned_sift/ready'

train_data_dir      = base_dir + '/patches/train'
validation_data_dir = base_dir + '/patches/validation'
test_data_dir       = base_dir + '/patches/test'

train_descrs_dir      = base_dir + '/descriptors_angles/train'
validation_descrs_dir = base_dir + '/descriptors_angles/validation'
test_descrs_dir       = base_dir + '/descriptors_angles/test'

In [4]:
def loading_data(dir_patches, dir_descrs):
    files_patches = listdir(dir_patches + '/class0')
    files_patches.sort()
    
    files_descrs = listdir(dir_descrs + '/class0')
    files_descrs.sort()
    
    assert len(files_patches) == len(files_descrs), "The number of patches doesn't match the number of descriptors."

    patches = []
    descrs = []

    
    for file_patch, file_descr in zip(files_patches, files_descrs):
        patch = imageio.imread(dir_patches + '/class0/' + file_patch)
#         print(patch.shape)
        if patch.shape[0] == 31:
            patches.append(patch)
            descr = np.load(dir_descrs + '/class0/' + file_descr)
            descrs.append(descr)
#         elif image.shape[0] == 19:
#             temp_count19 += 1
        
        

    patches = np.array(patches)
    patches = patches.astype(np.float64) / 255
    
    descrs = np.array(descrs)
    descrs = descrs.astype(np.float64) / 255
    print("patches", patches.shape, "  descrs", descrs.shape)
    
    return patches, descrs

In [5]:
x_train, y_train = loading_data(train_data_dir, train_descrs_dir)
x_validation, y_validation = loading_data(validation_data_dir, validation_descrs_dir)

patches (3382, 31, 31)   descrs (3382, 129)
patches (409, 31, 31)   descrs (409, 129)


In [6]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[2], x_train.shape[2], 1))
x_validation = x_validation.reshape((x_validation.shape[0], x_validation.shape[2], x_validation.shape[2], 1))

In [7]:
x_validation.shape

(409, 31, 31, 1)

In [15]:
last_layer_bias = np.median(y_train)

In [27]:
last_layer_bias

0.058823529411764705

In [26]:
def MSLE_plus_plus(y_true, y_pred):
    if not K.is_tensor(y_pred):
        y_pred = K.constant(y_pred)
    y_true = K.cast(y_true, y_pred.dtype)
    first_log = K.log(K.clip(y_pred, K.epsilon(), None) + 1.)
    second_log = K.log(K.clip(y_true, K.epsilon(), None) + 1.)
    custom_loss_log = K.log(K.clip(y_true + y_pred, K.epsilon(), None) + 1.)
    custom_loss_denominator = (y_true * y_pred + 0.005) * 256  # parameters to be further adjusted
    return K.mean(K.square(first_log - second_log) + custom_loss_log / custom_loss_denominator, axis=-1)

In [18]:
input_shape = (img_width, img_height, 1)
input_img = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation="elu", padding="valid")(input_img)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = Conv2D(32, (3, 3), activation="elu", padding="valid")(x)
x = Conv2D(16, (3, 3), activation="elu", padding="valid")(x)
x = MaxPooling2D((4, 4), padding="valid")(x)
x = Flatten(data_format="channels_last")(x)
encoded = Dense(129, activation="elu", bias_initializer=initializers.Constant(last_layer_bias))(x)

encoder = Model(input_img, encoded)

encoder.compile(optimizer='adadelta', loss="binary_crossentropy")
encoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 31, 31, 1)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 29, 29, 32)        320       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 27, 27, 32)        9248      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 25, 25, 32)        9248      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 23, 23, 32)        9248      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 21, 21, 16)        4624      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 16)          0   

In [19]:
def fixed_generator(x_train, y_train, batch_size):
    while True:
        batch_list_x = []
        batch_list_y = []
        
        for i in range(x_train.shape[0]):
            batch_list_x.append(x_train[i])
            batch_list_y.append(y_train[i])
            if len(batch_list_x) == batch_size:
                yield (np.array(batch_list_x),np.array(batch_list_y))
                batch_list_x = []
                batch_list_y = []


In [20]:
model_version = '2.0.0.3_encoder_elueverywhere_bce_adadelta_initbias'

os.system('mkdir ' + base_dir + '/weights' + model_version)
checkpointer = keras.callbacks.ModelCheckpoint(base_dir + '/weights' + model_version + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

encoder.fit_generator(fixed_generator(x_train, y_train, batch_size),
                steps_per_epoch=x_train.shape[0],
                epochs=50,
                validation_data=fixed_generator(x_validation, y_validation, batch_size),
                validation_steps=x_validation.shape[0],
                callbacks=[checkpointer]
                )
encoder.save(base_dir + '/ae' + model_version + '.h5')

Epoch 1/50
3382/3382 [==============================] - 72s 21ms/step - loss: 0.3399 - val_loss: 0.3343
Epoch 2/50
3382/3382 [==============================] - 72s 21ms/step - loss: 0.3346 - val_loss: 0.3322
Epoch 3/50
3382/3382 [==============================] - 72s 21ms/step - loss: 0.3336 - val_loss: 0.3379
Epoch 4/50
3382/3382 [==============================] - 72s 21ms/step - loss: 0.3329 - val_loss: 0.3406
Epoch 5/50
3382/3382 [==============================] - 72s 21ms/step - loss: 0.3323 - val_loss: 0.3363
Epoch 6/50
3382/3382 [==============================] - 72s 21ms/step - loss: 0.3316 - val_loss: 0.3236
Epoch 7/50
3382/3382 [==============================] - 72s 21ms/step - loss: 0.3311 - val_loss: 0.3289
Epoch 8/50
3382/3382 [==============================] - 72s 21ms/step - loss: 0.3306 - val_loss: 0.3423
Epoch 9/50
3382/3382 [==============================] - 72s 21ms/step - loss: 0.3303 - val_loss: 0.3199
Epoch 10/50
3382/3382 [==============================] - 72s 21m

In [22]:
np.around(encoder.predict(imageio.imread(test_data_dir + "/class0/patch000013.bmp").reshape(1,31,31,1))*255)

array([[255., 255., 255.,   0.,   0.,   0., 255., 255., 255., 255., 195.,
          0.,   0.,   0., 255.,   0., 255., 255.,   0., 255.,   0.,   0.,
          0.,   0.,   0.,   0.,   0., 255., 255., 255.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0., 255., 255., 255., 255.,   0.,   0.,
          0.,   0., 255.,   0., 255.,   6.,   0.,   0., 255.,   0.,   0.,
          0.,   0.,   0.,   0., 255., 255.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0., 255., 255.,   0.,   0.,   0.,   0.,   0.,
          0., 255., 255., 255.,   0.,   0.,   0.,   0., 255., 255., 255.,
        255.,   0.,   0.,   0., 255., 255., 255., 255.,   0.,   0., 255.,
        255.,   0.,   0.,   0.,   0.,   0., 255., 255., 255.,   0.,   0.,
          0.,   0., 255., 255., 255.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0., 255.,   0.,   0., 255.,   0., 255.]], dtype=float32)

In [23]:
np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch000001.bmp").reshape(1,31,31,1))*255)

array([[  47.,  459.,  299.,  156.,  151.,  562.,   78., -161.,  333.,
         759.,  353.,  279.,  361.,  306., -154., -197.,  821.,  628.,
          25.,   83.,  169.,  329., -226., -217.,  -45.,  157.,  -70.,
        -155.,  112.,  529., -108., -136.,   61.,  397.,  157.,  221.,
          64.,  244.,  -26.,   11.,  246.,  500., -169., -142.,  480.,
         552.,   69., -149.,  537.,  243., -146., -166.,    1.,   -5.,
         -11., -122.,  212.,   38., -147., -182., -156.,  163.,  -44.,
        -162.,  304., -200., -188.,  122.,  200.,   44.,   57., 1039.,
          85., -191., -219.,  -36.,  368.,  212.,  190.,  538.,  103.,
         -81., -167., -222.,  200.,  246.,   82.,   40.,  124.,   78.,
         -40.,  -17.,  -93.,   38., -156., -108.,  225.,  149.,  -56.,
        -115.,  160.,  403.,   33.,  294.,   49.,   16.,   70.,  340.,
         481.,  275., -205., -239.,  292.,  521.,   25.,   50.,  265.,
         202., -182., -217.,   21.,  223.,  243.,  -21.,  -44.,   22.,
      

In [24]:
np.load(train_descrs_dir + "/class0/descr_angle_000001.npy")

array([  4,   1,   0,   0, 137,  42,   1,   3,  55,   7,   0,   0,  71,
        32,   1,   3, 137,  39,   0,   0,   3,   2,   0,   5,  40,  10,
         1,  13,  81,   1,   0,   1,  11,   2,   0,   0, 137,  64,   1,
         1,  54,   2,   0,   0, 109,  23,   1,   4, 137,  33,   4,   2,
         8,   2,   0,   8,  40,  16,  12,  83,  83,   0,   0,   1,  27,
         9,   0,   1, 137,  58,   2,   0,  23,   0,   0,   0,  54,  62,
        17,  18, 137,  36,  12,   8,   5,  10,   9,  34,  10,  16,  22,
       137,  76,   2,   0,   0,  50,   8,   0,   1, 137,  57,   1,   1,
        26,   1,   0,   0,  31,  26,  18,  19,  19,   2,   4,   4,  10,
        10,  17,  47,   1,   1,   2,  27, 137,  25,   2,   3,  19],
      dtype=uint8)

In [25]:
temp_y_p1 = np.load(train_descrs_dir + "/class0/descr_angle_000001.npy")
temp_x_p1 = np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch000001.bmp").reshape(1,31,31,1))*255)
temp_x_p1 = temp_x_p1.reshape(129)

In [26]:
np.corrcoef(temp_x_p1, temp_y_p1)[0,1]

0.1924453450634157

In [28]:
corr_coefs = []
for i in range(103):
    try:
        temp_y_p1 = np.load(train_descrs_dir + "/class0/descr_angle_" + str(i).zfill(6) + ".npy")
        temp_x_p1 = np.around(encoder.predict(imageio.imread(train_data_dir + "/class0/patch" + str(i).zfill(6) + ".bmp").reshape(1,31,31,1))*255)
        temp_x_p1 = temp_x_p1.reshape(129)
        corr_coefs.append(np.corrcoef(temp_x_p1, temp_y_p1)[0,1])
    except:
        continue
corr_coefs = np.array(corr_coefs)
print(np.median(corr_coefs))

0.10761192494360794


In [45]:
temp_y_prime = encoder.predict(imageio.imread("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_train16/class0/patch000012.png").reshape(1,16,16,3))

In [46]:
temp_y = np.load("/home/niaki/Code/ImageNet/tiny-imagenet-200/tiny_sifts/tiny_train16/class0/patch000012.npy")

In [47]:
print(np.corrcoef(temp_y, temp_y_prime[0])[1,0], "\n")
for i in range(temp_y_prime.shape[1]):
    print('{:>12f}{:>12f}'.format(temp_y[i], temp_y_prime[0, i]))      
#     print(temp_y_prime[0, i], temp_y[i])

0.6393183138370047 

    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    0.000000    0.000000
    

In [21]:
np.round(temp, decimals=9)

array([[0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 3.427e-06, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+0

In [14]:
import tensorflow as tf

print(tf.__version__)

1.14.0


In [19]:
tf.disable_v2_behavior()

W0122 20:00:43.071549 140376228677440 deprecation.py:323] From /scratch/tensorflow/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:61: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [21]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)